# Importing some packages

In [1]:
import numpy as np
from numba import njit
import pandas as pd
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

# Limited testing

In [2]:
@njit(cache=True)
def get_signals(signal_list, exit_list):

    start_idx = 0
    exit_idx = 0
   
    for i in range(len(signal_list[0])):

        if i == start_idx:

            if signal_list[0][i] == 0:

                start_idx += 1

                exit_list[0][i] = 0

            else:

                for j in range(i+1, len(exit_list[0])):
                    if exit_list[0][j] == -signal_list[0][i]:
                        exit_idx = j
                        break
                    else:
                        exit_idx = j

                for k in range(i+1, exit_idx+1):
                    if signal_list[0][k] == -signal_list[0][i]:
                        exit_idx = k
                        exit_list[1][k] = signal_list[1][k]
                        break
                    else:
                        exit_idx = k

                for p in range(i+1, exit_idx):
                    signal_list[0][p] = 0
                    exit_list[0][p] = 0
                
                if exit_idx == len(signal_list[0]) - 1 and exit_list[0][exit_idx] != -signal_list[0][i]:
                    exit_list[0][exit_idx] = 0
                    exit_list[0][i] = 0
                    signal_list[0][exit_idx] = 0
                else:
                    exit_list[0][exit_idx] = -signal_list[0][i]
                    exit_list[0][i] = 0
                    signal_list[0][exit_idx] = 0
                
                start_idx = exit_idx + 1

        else:

            continue

    if sum(np.abs(signal_list[0])) == sum(np.abs(exit_list[0])):

        return signal_list, exit_list
    
    else:

        for i in range(len(signal_list[0])):
            if signal_list[0][-(i+1)] != 0:
                signal_list[0][-(i+1)] = 0
                break

        return signal_list, exit_list
    
@njit(cache=True)
def create_position_open_prices(signal_list, exit_list):

    pos_open_prices = np.zeros(len(signal_list[0]))
    pos_exit_prices = np.zeros(len(exit_list[0]))

    start_idx = 0
    price_idx = 0

    for i in range(len(signal_list[0])):
        if exit_list[0][i] != 0:
            for j in range(start_idx, i):
                if signal_list[0][j] != 0:
                    price_idx = j
                    break
            pos_open_prices[i] = signal_list[1][price_idx]
            pos_exit_prices[i] = exit_list[1][i]
            start_idx = i
        else:
            pass

    return pos_open_prices, pos_exit_prices

@njit(cache=True)
def get_pnl_testing(
    trade_close_prices,
    signal_list, 
    trade_open_prices,
    commission=0.015,
    slippage=0.05,
    init_inv=20000,
    trade_size=0.1
):

    pnl_list = np.zeros(len(trade_close_prices))

    for i in range(len(trade_close_prices)):

        if signal_list[i] == 0 or trade_open_prices[i] == 0:
            pass
        
        # signal_list contains the points where exit occurs
        elif signal_list[i] == -1: 
            temp_n_assets = int(init_inv * trade_size / trade_open_prices[i])
            temp_pnl = temp_n_assets * (trade_close_prices[i] - trade_open_prices[i] * (1 + slippage)) 
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

        else:
            temp_n_assets = int(init_inv * trade_size / trade_open_prices[i])
            temp_pnl = temp_n_assets * (trade_open_prices[i] * (1 - slippage) - trade_close_prices[i])
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

        pnl_list[i] = temp_pnl

    return pnl_list


## Entry testing

### Fixed stop and target exit

In [3]:
@njit(cache=True)
def get_exit_entry_testing1(
    close_prices, 
    open_prices,  
    signal_list,
    stoploss_th,
    takeprofit_th, 
    commission, 
    slippage, 
    init_inv, 
    trade_size
):

    exit_list = np.zeros((2, len(close_prices)))

    for i in range(len(close_prices)-1):

        if signal_list[1][i] == 0:

            pass

        else:

            temp_n_assets = int(init_inv * trade_size / signal_list[1][i])
            if signal_list[0][i] == 1:
                temp_pnl = temp_n_assets * (close_prices[i] - signal_list[1][i] * (1 + slippage))
            else:
                temp_pnl = -temp_n_assets * (close_prices[i] - signal_list[1][i] * (1 - slippage))
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

            if -temp_pnl >= stoploss_th or temp_pnl >= takeprofit_th:
                exit_list[0][i+1] = -signal_list[0][i]
                exit_list[1][i+1] = open_prices[i+1]
            else:
                pass
        
    return exit_list

### Fixed bar exit

In [4]:
@njit(cache=True)
def get_exit_entry_testing2( 
    open_prices,  
    signal_list,
    n_exit_bars
):

    exit_list = np.zeros((2, len(signal_list[0])))

    n_exit_bars = np.int64(n_exit_bars)

    for i in range(len(signal_list[0])-1):

        if signal_list[0][i] == 0:

            pass

        else:
            
            if i + n_exit_bars < len(signal_list[0]):
                exit_list[0][i+n_exit_bars] = -signal_list[0][i]
                exit_list[1][i+n_exit_bars] = open_prices[i+n_exit_bars]
            else:
                pass
        
    return exit_list


### Random exit

In [5]:
@njit(cache=True)
def get_exit_entry_testing3( 
    open_prices,  
    signal_list
):

    exit_list = np.zeros((2, len(signal_list[0])))

    for i in range(len(signal_list[0])-1):

        if signal_list[0][i] == 0:

            pass

        else:

            for j in range(i+1, len(signal_list[0])):
                if signal_list[0][j] != 0:
                    j = j - 1
                    break
                else:
                    if np.random.rand() > 0.5:
                        break
            
            exit_list[0][j] = -signal_list[0][i]
            exit_list[1][j] = open_prices[j]
        
    return exit_list


### Winning percentage

In [6]:
def calculate_mean_win_perc_entry_testing(data, text_code):

    bars_per_5week = 7 * 60 * 24 * 5
    n_bars_per_year = 7 * 60 * 24 * 52

    if data.shape[0] < n_bars_per_year:
        n_bars_per_year = data.shape[0]

    n_not_worked = 0
    n_total_cases = 0

    entry_walk_forward_dict = defaultdict(list)

    for idx in range(0, n_bars_per_year, bars_per_5week):

        n_total_cases += 1

        df = data.iloc[idx:idx+bars_per_5week, :]
        df.reset_index(drop=True, inplace=True)

        price_data = {}
        for col in df.columns:
            if col == 'datetime':
                continue
            else:
                price_data[col] = df[col].values
        
        try:
            exec_dict = {'price_data': price_data}
            exec(text_code, exec_dict)

            commission = exec_dict['COMMISSION']
            slippage = exec_dict['SLIPPAGE'] 
            init_inv = exec_dict['AVAILABLE_CAPITAL']
            trade_size = exec_dict['TRADE_SIZE'] 

            signal_idxs = list(exec_dict['buy_idxs'])
            signal_idxs.extend(list(exec_dict['sell_idxs']))
            signal_idxs = sorted(signal_idxs)
            signal_idxs_true = [i - 1 for i in signal_idxs]

            buy_idxs = [i - 1 for i in list(exec_dict['buy_idxs'])]
            df['buy'] = 0
            df.loc[df.index.isin(buy_idxs), 'buy'] = 1

            sell_idxs = [i - 1 for i in list(exec_dict['sell_idxs'])]
            df['sell'] = 0
            df.loc[df.index.isin(sell_idxs), 'sell'] = -1

            df['signal'] = df['buy'] + df['sell']

            df['new_signal'] = 0
            df.loc[df.index.isin(signal_idxs_true), 'new_signal'] = df.loc[df.index.isin(signal_idxs_true), 'signal'].values
            df['signal_prices'] = 0
            df.loc[df.index.isin(signal_idxs), 'signal_prices'] = df.loc[df.index.isin(signal_idxs), 'btc_open'].values

            signal_list = np.zeros((2, df.shape[0]))
            signal_list[0][1:] = df['new_signal'].values[:-1]
            signal_list[1] = df['signal_prices'].values

            # fixed stop and target exit testing
            exit_list = get_exit_entry_testing1(
                close_prices=df['btc_close'].values, 
                open_prices=df['btc_open'].values,  
                signal_list=signal_list,
                stoploss_th=50,
                takeprofit_th=100,
                commission=commission, 
                slippage=slippage, 
                init_inv=init_inv, 
                trade_size=trade_size
            )

            signal_list, exit_list = get_signals(signal_list, exit_list)
            pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

            pnl_list = get_pnl_testing(
                trade_close_prices=pos_exit_prices,
                signal_list=exit_list[0], 
                trade_open_prices=pos_open_prices,
                commission=commission, 
                slippage=slippage, 
                init_inv=init_inv, 
                trade_size=trade_size
            )

            fixed_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
            entry_walk_forward_dict['fixed_sp_testing'].append(fixed_winning_percent)

            # fixed bar exit testing
            exit_list = get_exit_entry_testing2( 
                open_prices=df['btc_open'].values,  
                signal_list=signal_list,
                n_exit_bars=5
            )

            signal_list, exit_list = get_signals(signal_list, exit_list)
            pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

            pnl_list = get_pnl_testing(
                trade_close_prices=pos_exit_prices,
                signal_list=exit_list[0], 
                trade_open_prices=pos_open_prices,
                commission=commission, 
                slippage=slippage, 
                init_inv=init_inv, 
                trade_size=trade_size
            )

            fixed_bar_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
            entry_walk_forward_dict['fixed_bar_testing'].append(fixed_bar_winning_percent)

            # random exit testing
            exit_list = get_exit_entry_testing3( 
                open_prices=df['btc_open'].values,  
                signal_list=signal_list
            )

            signal_list, exit_list = get_signals(signal_list, exit_list)
            pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

            pnl_list = get_pnl_testing(
                trade_close_prices=pos_exit_prices,
                signal_list=exit_list[0], 
                trade_open_prices=pos_open_prices,
                commission=commission, 
                slippage=slippage, 
                init_inv=init_inv, 
                trade_size=trade_size
            )

            random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
            entry_walk_forward_dict['random_exit_testing'].append(random_winning_percent)

        except:

            n_not_worked += 1

    mean_win_perc_dict = defaultdict(list)

    mean_win_perc_dict['Fixed_StopLoss_TakeProfit_testing'].\
        append(np.mean(entry_walk_forward_dict['fixed_sp_testing']))
    mean_win_perc_dict['Fixed_Bar_testing'].\
        append(np.mean(entry_walk_forward_dict['fixed_bar_testing']))
    mean_win_perc_dict['Random_Exit_testing'].\
        append(np.mean(entry_walk_forward_dict['random_exit_testing']))
    mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)
    
    win_pc_df = pd.DataFrame(mean_win_perc_dict)

    return win_pc_df


## Exit testing

### Similar approach entry

In [7]:
@njit(cache=True)
def get_entry_exit_testing1(
    close_prices,
    open_prices,
    n_bars
):

    signal_list = np.zeros((2, len(close_prices)))

    # n_bars = np.int64(n_bars)

    for i in range(n_bars, len(signal_list[0])):
            
        if close_prices[i - n_bars] - close_prices[i - 1] > 0:
            signal_list[0][i] = 1
            signal_list[1][i] = open_prices[i]
        elif close_prices[i - n_bars] - close_prices[i - 1] < 0:
           signal_list[0][i] = -1
           signal_list[1][i] = open_prices[i]
        else:
            pass
        
    return signal_list

@njit(cache=True)
def get_rsi(close_prices, prev_close_prices, length=14):
    # Create numpy arrays to store the gain/loss values
    gains = np.zeros(len(close_prices))
    losses = np.zeros(len(close_prices))

    # Iterate through the data frame and calculate the gain/loss for each period
    for i in range(1, len(close_prices)):
        change = close_prices[i] - prev_close_prices[i]
        if change > 0:
            gains[i] = change
        elif change < 0:
            losses[i] = abs(change)

    # Calculate the average gain and loss for each period
    avg_gains = np.zeros(len(close_prices))
    avg_losses = np.zeros(len(close_prices))
    for i in range(length, len(close_prices)):
        avg_gains[i] = np.mean(gains[i-length:i])
        avg_losses[i] = np.mean(losses[i-length:i])

    # Calculate the relative strength and RSI for each period
    rs = np.zeros(len(close_prices))
    rsi = np.zeros(len(close_prices))
    
    for i in range(len(close_prices)):
        if i+1 < length:
            rsi[i] = -999
        elif avg_losses[i] == 0:
            rs[i] = avg_gains[i]
            rsi[i] = 100
        else:
            rs[i] = avg_gains[i] / avg_losses[i]
            rsi[i] = 100 - (100 / (1 + rs[i]))

    return rsi

@njit(cache=True)
def get_entry_exit_testing2(
    close_prices, open_prices, prev_close_prices, rsi_window_size, rsi_threshold
):

    signal_list = np.zeros((2, len(close_prices)))

    rsi = get_rsi(close_prices, prev_close_prices, length=rsi_window_size)

    for i in range(len(close_prices)-1):

        if i < rsi_window_size - 1 or rsi[i] == -999:
            continue
       
        if rsi[i] < rsi_threshold:
            signal_list[0][i+1] = 1
            signal_list[1][i+1] = open_prices[i+1]
        elif rsi[i] > (100 - rsi_threshold):
            signal_list[0][i+1] = -1
            signal_list[1][i+1] = open_prices[i+1]
        else:
            pass

    return signal_list


### Random entry

In [8]:
@njit(cache=True)
def get_entry_exit_testing3(
    open_prices
):

    signal_list = np.zeros((2, len(open_prices)))

    for i in range(len(open_prices)-1):

        if np.random.rand() > 0.7:
            signal_list[0][i] = 1
            signal_list[1][i] = open_prices[i]
        elif np.random.rand() < 0.3:
            signal_list[0][i] = -1
            signal_list[1][i] = open_prices[i]
        else:
            pass

    return signal_list


### Winning percentage

In [9]:
def calculate_mean_win_perc_exit_testing(data, text_code):

    bars_per_5week = 7 * 60 * 24 * 5
    n_bars_per_year = 7 * 60 * 24 * 52

    if data.shape[0] < n_bars_per_year:
        n_bars_per_year = data.shape[0]

    n_not_worked = 0
    n_total_cases = 0

    exit_walk_forward_dict = defaultdict(list)

    for idx in range(0, n_bars_per_year, bars_per_5week):

        n_total_cases += 1

        df = data.iloc[idx:idx+bars_per_5week, :]
        df.reset_index(drop=True, inplace=True)

        price_data = {}
        for col in df.columns:
            if col == 'datetime':
                continue
            else:
                price_data[col] = df[col].values

        try:

            exec_dict = {'price_data': price_data}
            exec(text_code, exec_dict)

            commission = exec_dict['COMMISSION']
            slippage = exec_dict['SLIPPAGE'] 
            init_inv = exec_dict['AVAILABLE_CAPITAL']
            trade_size = exec_dict['TRADE_SIZE']       

            signal_idxs = list(exec_dict['buy_idxs'])
            signal_idxs.extend(list(exec_dict['sell_idxs']))
            signal_idxs = sorted(signal_idxs)
            signal_idxs_true = [i - 1 for i in signal_idxs]

            buy_idxs = [i - 1 for i in list(exec_dict['buy_idxs'])]
            df['buy'] = 0
            df.loc[df.index.isin(buy_idxs), 'buy'] = 1

            sell_idxs = [i - 1 for i in list(exec_dict['sell_idxs'])]
            df['sell'] = 0
            df.loc[df.index.isin(sell_idxs), 'sell'] = -1

            df['signal'] = df['buy'] + df['sell']

            df['exit_signal'] = 0
            df.loc[df.index.isin(signal_idxs_true[1:]), 'exit_signal'] = df.loc[df.index.isin(signal_idxs_true[1:]), 'signal'].values
            df['exit_prices'] = 0
            df.loc[df.index.isin(signal_idxs[1:]), 'exit_prices'] = df.loc[df.index.isin(signal_idxs[1:]), 'btc_open'].values

            exit_list = np.zeros((2, df.shape[0]))
            exit_list[0][1:] = df['exit_signal'].values[:-1]
            exit_list[1] = df['exit_prices'].values

            # replacing entry with trend following entry
            signal_list = get_entry_exit_testing1(
                close_prices=df['btc_close'].values,
                open_prices=df['btc_open'].values,
                n_bars=5
            )

            signal_list, exit_list = get_signals(signal_list, exit_list)
            pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

            pnl_list = get_pnl_testing(
                trade_close_prices=pos_exit_prices,
                signal_list=exit_list[0], 
                trade_open_prices=pos_open_prices,
                commission=commission, 
                slippage=slippage, 
                init_inv=init_inv, 
                trade_size=trade_size
            )

            fixed_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
            exit_walk_forward_dict['trend_entry_testing'].append(fixed_winning_percent)

            # replacing entry with countertrend entry
            signal_list = get_entry_exit_testing2(
                close_prices=df['btc_close'].values, 
                open_prices=df['btc_open'].values, 
                prev_close_prices=df['btc_close'].shift(1).fillna(method='bfill').values, 
                rsi_window_size=10, 
                rsi_threshold = 20
            )

            signal_list, exit_list = get_signals(signal_list, exit_list)
            pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

            pnl_list = get_pnl_testing(
                trade_close_prices=pos_exit_prices,
                signal_list=exit_list[0], 
                trade_open_prices=pos_open_prices,
                commission=commission, 
                slippage=slippage, 
                init_inv=init_inv, 
                trade_size=trade_size
            )

            fixed_bar_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
            exit_walk_forward_dict['countertrend_entry_testing'].append(fixed_bar_winning_percent)

            # random entry testing
            signal_list = get_entry_exit_testing3(
                open_prices=df['btc_open'].values
            )

            signal_list, exit_list = get_signals(signal_list, exit_list)
            pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

            pnl_list = get_pnl_testing(
                trade_close_prices=pos_exit_prices,
                signal_list=exit_list[0], 
                trade_open_prices=pos_open_prices,
                commission=commission, 
                slippage=slippage, 
                init_inv=init_inv, 
                trade_size=trade_size
            )

            random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
            exit_walk_forward_dict['random_entry_testing'].append(random_winning_percent)

        except:

            n_not_worked += 1

    mean_win_perc_dict = defaultdict(list)

    mean_win_perc_dict['Trend_testing'].\
        append(np.mean(exit_walk_forward_dict['trend_entry_testing']))
    mean_win_perc_dict['Countertrend_testing'].\
        append(np.mean(exit_walk_forward_dict['countertrend_entry_testing']))
    mean_win_perc_dict['Random_Entry_testing'].\
        append(np.mean(exit_walk_forward_dict['random_entry_testing']))
    mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)
    
    win_pc_df = pd.DataFrame(mean_win_perc_dict)

    return win_pc_df


## Core testing

### Winning percentage

In [10]:
def calculate_mean_win_perc_core_testing(data, text_code):

    bars_per_5week = 7 * 60 * 24 * 5
    n_bars_per_year = 7 * 60 * 24 * 52

    if data.shape[0] < n_bars_per_year:
        n_bars_per_year = data.shape[0]

    n_not_worked = 0
    n_total_cases = 0

    core_walk_forward_dict = defaultdict(list)

    for idx in range(0, n_bars_per_year, bars_per_5week):

        n_total_cases += 1

        df = data.iloc[idx:idx+bars_per_5week, :]
        df.reset_index(drop=True, inplace=True)

        price_data = {}
        for col in df.columns:
            if col == 'datetime':
                continue
            else:
                price_data[col] = df[col].values

        try:
            exec_dict = {'price_data': price_data}
            exec(text_code, exec_dict)
            pnl_list = exec_dict['all_arr']

            winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
            core_walk_forward_dict['core_testing'].append(winning_percent)
        except:
            n_not_worked += 1

    mean_win_perc_dict = defaultdict(list)
    mean_win_perc_dict['Core_Testing'].\
        append(np.mean(core_walk_forward_dict['core_testing']))
    mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)
    
    win_pc_df = pd.DataFrame(mean_win_perc_dict)

    return win_pc_df


### Performance

In [11]:
# @njit(cache=True)
# def get_drawdown(pnl_list):

#     # running_total = np.cumsum(pnl_list)
#     # max_running_total = np.maximum.accumulate(running_total)

#     running_total = np.zeros(len(pnl_list))
#     max_running_total = np.zeros(len(pnl_list))

#     for i in range(len(pnl_list)):
#         if i == 0:
#             running_total[i] = pnl_list[i]
#             max_running_total[i] = pnl_list[i]
#         else:
#             running_total[i] = running_total[i-1] + pnl_list[i]
#             max_running_total[i] = max([max_running_total[i-1], pnl_list[i]])

#     min_total = running_total[np.argmin(running_total)]

#     max_total = max_running_total[np.argmin(running_total)]

#     if max_total == 0:
#         max_total = 0.00001

#     max_drawdown = 100 * (min_total - max_total) / max_total

#     if max_drawdown == -1:
#         max_drawdown = 0

#     return max_drawdown

@njit(cache=True)
def get_drawdown(pnl_list):
    max_dd = 0.0
    peak = pnl_list[0]

    for i in range(1, len(pnl_list)):
        if pnl_list[i] > peak:
            peak = pnl_list[i]
        drawdown = 100 * (peak - pnl_list[i]) / peak
        if drawdown > max_dd:
            max_dd = drawdown

    return max_dd

@njit(cache=True)
def get_sharpe_ratio(
    pnl_list, 
    risk_free_rate=0
):
    # mean_return = np.mean(pnl_list)
    mean_return = 0
    for i in range(len(pnl_list)):
        mean_return += pnl_list[i]

    if len(pnl_list) == 0:
        mean_return = 0
    else:
        mean_return = mean_return / len(pnl_list)

    # std_dev = np.std(pnl_list)
    std_dev = 0
    for i in range(len(pnl_list)):
        std_dev += (pnl_list[i] - mean_return) ** 2

    if len(pnl_list) == 1:
        std_dev = np.sqrt(std_dev/len(pnl_list))
    else:
        std_dev = np.sqrt(std_dev/(len(pnl_list) - 1))

    if std_dev == 0:
        std_dev += 0.0001

    sharpe_ratio = (mean_return - risk_free_rate) / std_dev
    return sharpe_ratio


In [12]:
def calculate_mean_performance(data, text_code):

    bars_per_5week = 7 * 60 * 24 * 5
    n_bars_per_year = 7 * 60 * 24 * 52

    if data.shape[0] < n_bars_per_year:
        n_bars_per_year = data.shape[0]

    n_not_worked = 0
    n_total_cases = 0

    performance_walk_forward_dict = defaultdict(list)

    for idx in range(0, n_bars_per_year, bars_per_5week):

        n_total_cases += 1

        df = data.iloc[idx:idx+bars_per_5week, :]
        df.reset_index(drop=True, inplace=True)

        price_data = {}
        for col in df.columns:
            if col == 'datetime':
                continue
            else:
                price_data[col] = df[col].values

        try:
            exec_dict = {'price_data': price_data}
            exec(text_code, exec_dict)
            pnl_list = exec_dict['all_arr']

            init_inv = exec_dict['AVAILABLE_CAPITAL']
            trade_size = exec_dict['TRADE_SIZE']       

            signal_idxs = list(exec_dict['buy_idxs'])
            signal_idxs.extend(list(exec_dict['sell_idxs']))
            signal_idxs = sorted(signal_idxs)

            performance_walk_forward_dict['n_trades'].append(len(signal_idxs) - 1)
            performance_walk_forward_dict['pnl'].append(np.sum(pnl_list))
            performance_walk_forward_dict['roi'].append(100 * np.sum(pnl_list) / (trade_size * init_inv))
            performance_walk_forward_dict['avg_drawdown'].append(exec_dict['avg_drawdown'])
            performance_walk_forward_dict['drawdown'].append(get_drawdown(pnl_list))
            performance_walk_forward_dict['pnl_avgd_ratio'].append(exec_dict['fitness'])
            annualized_sharpe_ration = np.sqrt(525600) * get_sharpe_ratio(pnl_list, risk_free_rate=0)
            performance_walk_forward_dict['sharpe_ratio'].append(annualized_sharpe_ration)
        except:
            n_not_worked += 1

    mean_perf_dict = defaultdict(list)
    mean_perf_dict['N_Trades'].\
        append(np.mean(performance_walk_forward_dict['n_trades']))
    mean_perf_dict['PNL'].\
        append(np.mean(performance_walk_forward_dict['pnl']))
    mean_perf_dict['ROI (%)'].\
        append(np.mean(performance_walk_forward_dict['roi']))
    mean_perf_dict['AVG_Drawdown'].\
        append(np.mean(performance_walk_forward_dict['avg_drawdown']))
    mean_perf_dict['Drawdown (%)'].\
        append(np.mean(performance_walk_forward_dict['drawdown']))
    mean_perf_dict['PNL_AVGD_Ratio'].\
        append(np.mean(performance_walk_forward_dict['pnl_avgd_ratio']))
    mean_perf_dict['Sharpe_Ratio'].\
        append(np.mean(performance_walk_forward_dict['sharpe_ratio']))
    mean_perf_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)
    
    perf_df = pd.DataFrame(mean_perf_dict)

    return perf_df


# Data loading function

In [13]:
def generate_52w_data(data_path):
    df = pd.read_csv(data_path)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.iloc[-(7 * 60 * 24 * 52):]
    df.sort_values('datetime', ascending=True, inplace=True)
    df.reset_index(inplace=True, drop=True)

    return df

# Testing strategies constructed from combination with other instruments

## Loading price and volume data

In [14]:
data_path = Path(r'C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\datasets/\all_data_1min.csv')
df_52w = generate_52w_data(data_path)
df_52w.head()

,datetime,6e_open,6e_high,6e_low,6e_close,6e_volume,aapl_open,aapl_high,aapl_low,aapl_close,...,zf_open,zf_high,zf_low,zf_close,zf_volume,zn_open,zn_high,zn_low,zn_close,zn_volume
0,2023-11-01 18:56:00,1.05600,1.05635,1.05590,1.05590,607,172.77,172.97,172.765,172.95,...,104.914062,104.945312,104.914062,104.937500,3727,106.812500,106.859375,106.796875,106.843750,11709
1,2023-11-01 18:58:00,1.05625,1.05640,1.05615,1.05630,396,173.10,173.27,173.060,173.25,...,104.937500,104.945312,104.921875,104.937500,3415,106.843750,106.859375,106.828125,106.859375,8757
2,2023-11-01 18:59:00,1.05630,1.05650,1.05605,1.05615,1569,173.24,173.24,173.090,173.19,...,104.937500,104.960938,104.929688,104.960938,13873,106.859375,106.875000,106.828125,106.875000,33414
3,2023-11-01 19:02:00,1.05650,1.05670,1.05640,1.05660,326,173.59,173.75,173.550,173.74,...,104.976562,105.015625,104.968750,105.007812,5523,106.906250,106.937500,106.890625,106.921875,7998
4,2023-11-01 19:03:00,1.05655,1.05675,1.05645,1.05665,327,173.72,173.73,173.650,173.72,...,105.000000,105.023438,105.000000,105.000000,4571,106.921875,106.953125,106.906250,106.906250,9335


## Loading strategies generated

In [15]:
strategy_file_path = Path(r'C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\derived_strategies/\ge_results_inst_ind_comb_numba.csv')

df_str = pd.read_csv(strategy_file_path)
df_str = df_str[(df_str['fitness'] < 0) & (df_str['fitness'] < -3) & (df_str['fitness'] > -50)]
df_str.sort_values('fitness', ascending=True, inplace=True)
df_str.reset_index(drop=True, inplace=True)
df_str

,buy,sell,fitness
0,"(get_lag(price_data['aav_volume'], lag=5)[MAX_...",np.cos(price_data['nvda_close'][MAX_LAG:]) > n...,-49.822877
1,"(get_lag(price_data['aav_volume'], lag=5)[MAX_...",np.cos(price_data['nvda_close'][MAX_LAG:]) > n...,-49.822877
2,"(get_lag(price_data['aav_volume'], lag=5)[MAX_...",np.cos(price_data['nvda_close'][MAX_LAG:]) > n...,-49.822877
3,"get_lag(price_data['btc_low'], lag=1)[MAX_LAG:...","get_lag(price_data['msft_volume'], lag=4)[MAX_...",-49.458332
4,(price_data['fet_open'][MAX_LAG:] + get_lag(pr...,price_data['ng_open'][MAX_LAG:] < numba_indica...,-49.351696
...,...,...,...
602,"np.cos(get_lag(price_data['nvda_low'], lag=5)[...","(np.sin(get_lag(price_data['amzn_high'], lag=2...",-3.217634
603,"(get_lag(price_data['nq_volume'], lag=5)[MAX_L...","get_lag(price_data['msft_volume'], lag=5)[MAX_...",-3.142318
604,"(np.sin(get_lag(price_data['sui_high'], lag=4)...","(np.sin(get_lag(price_data['cl_high'], lag=4)[...",-3.097268
605,((numba_indicators.n_day_high(prices=price_dat...,"(np.cos(get_lag(price_data['aav_low'], lag=1)[...",-3.078844


## Running the tests over the strategies

In [16]:
final_entry_win_pc_df = pd.DataFrame()
final_exit_win_pc_df = pd.DataFrame()
final_core_win_pc_df = pd.DataFrame()
final_perf_df = pd.DataFrame()

lag_txt = '{i}'

strategy_idx = 1

for row in tqdm(df_str.itertuples()):#df_str.iloc[:10].itertuples()

    buy_signal_txt = row.buy
    sell_signal_txt = row.sell

    txt_code = f'''import os
CUR_DIR = os.getcwd()
os.chdir('src')
#import pandas as pd
import numpy as np
import gc
from fitness.indicators import numba_indicators
from fitness.performance.helper_func import merge_pnl, get_drawdowns, get_pnl, trading_signals, get_lag
os.chdir(CUR_DIR)
#from numba import njit
COMMISSION = 0.015
SLIPPAGE = 0.00005
AVAILABLE_CAPITAL = 700000
TRADE_SIZE = 0.5
MAX_LAG = 5
buy_idxs, sell_idxs = trading_signals(buy_signal={buy_signal_txt}, sell_signal={sell_signal_txt})
if len(buy_idxs) == 0 or len(sell_idxs) == 0:
    fitness = -999
else:
    buy_idxs = np.array(buy_idxs)
    sell_idxs = np.array(sell_idxs)
    open_prices = price_data['btc_open']
    buy_prices = open_prices[np.isin(np.arange(len(open_prices)), buy_idxs)]
    sell_prices = open_prices[np.isin(np.arange(len(open_prices)), sell_idxs)]
    if buy_idxs[0] < sell_idxs[0]:
        buy_arr = get_pnl(sell_prices, buy_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 1)
        buy_pnl = np.sum(buy_arr)
        sell_arr = get_pnl(buy_prices[1:], sell_prices[:-1], COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 0)
        sell_pnl = np.sum(sell_arr)
        all_arr = merge_pnl(buy_arr, sell_arr)
    else:
        sell_arr = get_pnl(buy_prices, sell_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 0)
        sell_pnl = np.sum(sell_arr)
        buy_arr = get_pnl(sell_prices[1:], buy_prices[:-1], COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 1)
        buy_pnl = np.sum(buy_arr)
        all_arr = merge_pnl(sell_arr, buy_arr)
    total_pnl = buy_pnl + sell_pnl
    equity_curve_arr = np.cumsum(all_arr)
    drawdowns = get_drawdowns(equity_curve_arr)
    avg_drawdown = np.sum(drawdowns[drawdowns!=0]) / len(drawdowns[drawdowns!=0])
    fitness = total_pnl / avg_drawdown
    if np.isnan(fitness) or total_pnl <= 0 or len(drawdowns[drawdowns!=0]) == 0:
        fitness = -999
gc.collect()'''

    entry_win_pc_df = calculate_mean_win_perc_entry_testing(
        data=df_52w, 
        text_code=txt_code
    )

    temp_signal_df = pd.DataFrame({'strategy': f'strategy{strategy_idx}', 'buy': [buy_signal_txt], 'sell': [sell_signal_txt]})
    entry_win_pc_df = pd.concat([temp_signal_df, entry_win_pc_df], axis=1)
    final_entry_win_pc_df = pd.concat([final_entry_win_pc_df, entry_win_pc_df])

    exit_win_pc_df = calculate_mean_win_perc_exit_testing(
        data=df_52w, 
        text_code=txt_code
    )

    exit_win_pc_df = pd.concat([temp_signal_df, exit_win_pc_df], axis=1)
    final_exit_win_pc_df = pd.concat([final_exit_win_pc_df, exit_win_pc_df])

    core_win_pc_df = calculate_mean_win_perc_core_testing(
        data=df_52w, 
        text_code=txt_code
    )

    core_win_pc_df = pd.concat([temp_signal_df, core_win_pc_df], axis=1)
    final_core_win_pc_df = pd.concat([final_core_win_pc_df, core_win_pc_df])

    perf_df = calculate_mean_performance(
        data=df_52w, 
        text_code=txt_code
    )

    perf_df = pd.concat([temp_signal_df, perf_df], axis=1)
    final_perf_df = pd.concat([final_perf_df, perf_df])

    strategy_idx += 1

607it [08:18,  1.22it/s]


In [17]:
final_entry_win_pc_df.reset_index(drop=True, inplace=True)
final_exit_win_pc_df.reset_index(drop=True, inplace=True)
final_core_win_pc_df.reset_index(drop=True, inplace=True)
final_perf_df.reset_index(drop=True, inplace=True)

## Showing the entry testing results

In [18]:
final_entry_win_pc_df

,strategy,buy,sell,Fixed_StopLoss_TakeProfit_testing,Fixed_Bar_testing,Random_Exit_testing,Not_Working
0,strategy1,"(get_lag(price_data['aav_volume'], lag=5)[MAX_...",np.cos(price_data['nvda_close'][MAX_LAG:]) > n...,44.680022,51.810482,49.729962,0.0
1,strategy2,"(get_lag(price_data['aav_volume'], lag=5)[MAX_...",np.cos(price_data['nvda_close'][MAX_LAG:]) > n...,44.680022,51.810482,47.301886,0.0
2,strategy3,"(get_lag(price_data['aav_volume'], lag=5)[MAX_...",np.cos(price_data['nvda_close'][MAX_LAG:]) > n...,44.680022,51.810482,49.565306,0.0
3,strategy4,"get_lag(price_data['btc_low'], lag=1)[MAX_LAG:...","get_lag(price_data['msft_volume'], lag=4)[MAX_...",42.935750,53.214819,44.544004,0.0
4,strategy5,(price_data['fet_open'][MAX_LAG:] + get_lag(pr...,price_data['ng_open'][MAX_LAG:] < numba_indica...,56.963734,85.639767,18.556701,0.0
...,...,...,...,...,...,...,...
602,strategy603,"np.cos(get_lag(price_data['nvda_low'], lag=5)[...","(np.sin(get_lag(price_data['amzn_high'], lag=2...",41.472147,43.271887,52.543066,0.0
603,strategy604,"(get_lag(price_data['nq_volume'], lag=5)[MAX_L...","get_lag(price_data['msft_volume'], lag=5)[MAX_...",52.336927,54.718216,51.179202,0.0
604,strategy605,"(np.sin(get_lag(price_data['sui_high'], lag=4)...","(np.sin(get_lag(price_data['cl_high'], lag=4)[...",27.329256,45.261328,28.546299,0.0
605,strategy606,((numba_indicators.n_day_high(prices=price_dat...,"(np.cos(get_lag(price_data['aav_low'], lag=1)[...",42.348995,52.134602,45.384442,0.0


### Finding strategies that have winning percentage equal to 60 or more for entry testing cases

In [19]:
entry_testing_strategies = final_entry_win_pc_df[
    (final_entry_win_pc_df['Fixed_StopLoss_TakeProfit_testing'] >= 60) & 
    (final_entry_win_pc_df['Fixed_Bar_testing'] >= 60) & 
    (final_entry_win_pc_df['Random_Exit_testing'] >= 60)
]['strategy'].tolist()
final_entry_win_pc_df[final_entry_win_pc_df['strategy'].isin(entry_testing_strategies)]

,strategy,buy,sell,Fixed_StopLoss_TakeProfit_testing,Fixed_Bar_testing,Random_Exit_testing,Not_Working
5,strategy6,"(get_lag(price_data['zf_volume'], lag=2)[MAX_L...","np.sin(get_lag(price_data['sui_open'], lag=2)[...",71.544265,78.462169,81.828061,0.0
17,strategy18,(price_data['es_volume'][MAX_LAG:] / 0.28371) ...,price_data['tia_high'][MAX_LAG:] < 6,80.956650,89.559754,90.199128,0.0
35,strategy36,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",95.616119,90.157392,98.968545,0.0
36,strategy37,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",95.616119,90.157392,98.962363,0.0
37,strategy38,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",95.616119,90.157392,98.964452,0.0
38,strategy39,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",95.616119,90.157392,98.677882,0.0
39,strategy40,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",95.616119,90.157392,98.825287,0.0
40,strategy41,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",95.616119,90.157392,98.935800,0.0
41,strategy42,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",95.616119,90.157392,98.675863,0.0
42,strategy43,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",95.616119,90.157392,98.327978,0.0


## Showing exit testing results

In [20]:
final_exit_win_pc_df

,strategy,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working
0,strategy1,"(get_lag(price_data['aav_volume'], lag=5)[MAX_...",np.cos(price_data['nvda_close'][MAX_LAG:]) > n...,59.429659,60.784514,49.878286,0.0
1,strategy2,"(get_lag(price_data['aav_volume'], lag=5)[MAX_...",np.cos(price_data['nvda_close'][MAX_LAG:]) > n...,59.429659,60.784514,49.067256,0.0
2,strategy3,"(get_lag(price_data['aav_volume'], lag=5)[MAX_...",np.cos(price_data['nvda_close'][MAX_LAG:]) > n...,59.429659,60.784514,49.713643,0.0
3,strategy4,"get_lag(price_data['btc_low'], lag=1)[MAX_LAG:...","get_lag(price_data['msft_volume'], lag=4)[MAX_...",59.832662,61.195801,51.046248,0.0
4,strategy5,(price_data['fet_open'][MAX_LAG:] + get_lag(pr...,price_data['ng_open'][MAX_LAG:] < numba_indica...,60.339995,60.424383,49.221674,0.0
...,...,...,...,...,...,...,...
602,strategy603,"np.cos(get_lag(price_data['nvda_low'], lag=5)[...","(np.sin(get_lag(price_data['amzn_high'], lag=2...",59.961541,60.696528,49.471230,0.0
603,strategy604,"(get_lag(price_data['nq_volume'], lag=5)[MAX_L...","get_lag(price_data['msft_volume'], lag=5)[MAX_...",60.106786,60.852039,49.455921,0.0
604,strategy605,"(np.sin(get_lag(price_data['sui_high'], lag=4)...","(np.sin(get_lag(price_data['cl_high'], lag=4)[...",60.292944,60.258640,50.413189,0.0
605,strategy606,((numba_indicators.n_day_high(prices=price_dat...,"(np.cos(get_lag(price_data['aav_low'], lag=1)[...",59.761062,60.794745,50.055226,0.0


### Showing strategies that have passed entry testing

In [21]:
final_exit_win_pc_df[final_exit_win_pc_df['strategy'].isin(entry_testing_strategies)]

,strategy,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working
5,strategy6,"(get_lag(price_data['zf_volume'], lag=2)[MAX_L...","np.sin(get_lag(price_data['sui_open'], lag=2)[...",60.358407,60.430521,50.055226,0.0
17,strategy18,(price_data['es_volume'][MAX_LAG:] / 0.28371) ...,price_data['tia_high'][MAX_LAG:] < 6,60.335904,60.450983,49.848634,0.0
35,strategy36,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",60.319538,60.418244,49.650221,0.0
36,strategy37,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",60.319538,60.418244,49.500900,0.0
37,strategy38,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",60.319538,60.418244,48.129283,0.0
38,strategy39,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",60.319538,60.418244,48.789118,0.0
39,strategy40,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",60.319538,60.418244,48.419890,0.0
40,strategy41,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",60.319538,60.418244,49.316806,0.0
41,strategy42,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",60.319538,60.418244,50.095125,0.0
42,strategy43,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",60.319538,60.418244,48.401023,0.0


### Showing strategies that have winning percentage equal to 60 or more (except for Random_Entry_testing)

In [22]:
exit_testing_strategies = final_exit_win_pc_df[
    (final_exit_win_pc_df['Trend_testing'] >= 60) & 
    (final_exit_win_pc_df['Countertrend_testing'] >= 60) & 
    (final_exit_win_pc_df['Random_Entry_testing'] >= 50) &
    (final_exit_win_pc_df['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

final_exit_win_pc_df[final_exit_win_pc_df['strategy'].isin(exit_testing_strategies)]

,strategy,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working
5,strategy6,"(get_lag(price_data['zf_volume'], lag=2)[MAX_L...","np.sin(get_lag(price_data['sui_open'], lag=2)[...",60.358407,60.430521,50.055226,0.0
41,strategy42,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",60.319538,60.418244,50.095125,0.0
44,strategy45,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",60.319538,60.418244,50.458265,0.0
80,strategy81,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",60.323630,60.430521,50.024546,0.0
125,strategy126,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",60.333858,60.426429,50.175934,0.0
293,strategy294,"((np.cos(get_lag(price_data['lin_open'], lag=1...","np.cos(get_lag(price_data['googl_open'], lag=3...",60.119060,60.393690,50.005115,0.0
486,strategy487,"np.cos(get_lag(price_data['nvda_low'], lag=5)[...","(np.sqrt(get_lag(price_data['amzn_close'], lag...",60.221345,60.375274,50.315044,0.0
493,strategy494,"get_lag(price_data['aapl_volume'], lag=5)[MAX_...","get_lag(price_data['dog_volume'], lag=2)[MAX_L...",60.352270,60.432567,50.048068,0.0


## Showing core testing results

In [23]:
final_core_win_pc_df

,strategy,buy,sell,Core_Testing,Not_Working
0,strategy1,"(get_lag(price_data['aav_volume'], lag=5)[MAX_...",np.cos(price_data['nvda_close'][MAX_LAG:]) > n...,74.989833,0.0
1,strategy2,"(get_lag(price_data['aav_volume'], lag=5)[MAX_...",np.cos(price_data['nvda_close'][MAX_LAG:]) > n...,74.989833,0.0
2,strategy3,"(get_lag(price_data['aav_volume'], lag=5)[MAX_...",np.cos(price_data['nvda_close'][MAX_LAG:]) > n...,74.989833,0.0
3,strategy4,"get_lag(price_data['btc_low'], lag=1)[MAX_LAG:...","get_lag(price_data['msft_volume'], lag=4)[MAX_...",25.137471,0.0
4,strategy5,(price_data['fet_open'][MAX_LAG:] + get_lag(pr...,price_data['ng_open'][MAX_LAG:] < numba_indica...,45.454545,0.0
...,...,...,...,...,...
602,strategy603,"np.cos(get_lag(price_data['nvda_low'], lag=5)[...","(np.sin(get_lag(price_data['amzn_high'], lag=2...",24.320000,0.0
603,strategy604,"(get_lag(price_data['nq_volume'], lag=5)[MAX_L...","get_lag(price_data['msft_volume'], lag=5)[MAX_...",75.102041,0.0
604,strategy605,"(np.sin(get_lag(price_data['sui_high'], lag=4)...","(np.sin(get_lag(price_data['cl_high'], lag=4)[...",73.877551,0.0
605,strategy606,((numba_indicators.n_day_high(prices=price_dat...,"(np.cos(get_lag(price_data['aav_low'], lag=1)[...",73.923739,0.0


### Showing strategies that has passed entry testing and have winning percentage equal to 60 or more

In [24]:
core_testing_strategies = final_core_win_pc_df[
    (final_core_win_pc_df['Core_Testing'] >= 60) &
    (final_core_win_pc_df['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

final_core_win_pc_df[final_core_win_pc_df['strategy'].isin(core_testing_strategies)]

,strategy,buy,sell,Core_Testing,Not_Working
5,strategy6,"(get_lag(price_data['zf_volume'], lag=2)[MAX_L...","np.sin(get_lag(price_data['sui_open'], lag=2)[...",76.470588,0.0
35,strategy36,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",69.230769,0.0
36,strategy37,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",69.230769,0.0
37,strategy38,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",69.230769,0.0
38,strategy39,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",69.230769,0.0
39,strategy40,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",69.230769,0.0
40,strategy41,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",69.230769,0.0
41,strategy42,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",69.230769,0.0
42,strategy43,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",69.230769,0.0
43,strategy44,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",69.230769,0.0


### Showing strategies that has passed both entry and exit testing and have winning percentage equal to 60 or more

In [25]:
core_testing_strategies2 = final_core_win_pc_df[
    (final_core_win_pc_df['Core_Testing'] >= 60) &
    (final_core_win_pc_df['strategy'].isin(exit_testing_strategies))
]['strategy'].tolist()

final_core_win_pc_df[final_core_win_pc_df['strategy'].isin(core_testing_strategies2)]

,strategy,buy,sell,Core_Testing,Not_Working
5,strategy6,"(get_lag(price_data['zf_volume'], lag=2)[MAX_L...","np.sin(get_lag(price_data['sui_open'], lag=2)[...",76.470588,0.0
41,strategy42,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",69.230769,0.0
44,strategy45,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",69.230769,0.0
125,strategy126,(price_data['es_volume'][MAX_LAG:] / 0.28771) ...,"(np.sqrt(get_lag(price_data['tia_close'], lag=...",69.230769,0.0
293,strategy294,"((np.cos(get_lag(price_data['lin_open'], lag=1...","np.cos(get_lag(price_data['googl_open'], lag=3...",78.534031,0.0
486,strategy487,"np.cos(get_lag(price_data['nvda_low'], lag=5)[...","(np.sqrt(get_lag(price_data['amzn_close'], lag...",72.972973,0.0
493,strategy494,"get_lag(price_data['aapl_volume'], lag=5)[MAX_...","get_lag(price_data['dog_volume'], lag=2)[MAX_L...",85.714286,0.0


## Showing performance

In [26]:
final_perf_df

,strategy,buy,sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),PNL_AVGD_Ratio,Sharpe_Ratio,Not_Working
0,strategy1,"(get_lag(price_data['aav_volume'], lag=5)[MAX_...",np.cos(price_data['nvda_close'][MAX_LAG:]) > n...,2459.0,4.699015e+05,134.257574,2383.407030,261.000819,49.822877,35.614817,0.0
1,strategy2,"(get_lag(price_data['aav_volume'], lag=5)[MAX_...",np.cos(price_data['nvda_close'][MAX_LAG:]) > n...,2459.0,4.699015e+05,134.257574,2383.407030,261.000819,49.822877,35.614817,0.0
2,strategy3,"(get_lag(price_data['aav_volume'], lag=5)[MAX_...",np.cos(price_data['nvda_close'][MAX_LAG:]) > n...,2459.0,4.699015e+05,134.257574,2383.407030,261.000819,49.822877,35.614817,0.0
3,strategy4,"get_lag(price_data['btc_low'], lag=1)[MAX_LAG:...","get_lag(price_data['msft_volume'], lag=4)[MAX_...",1273.0,1.126222e+04,3.217778,3944.213512,273.275146,49.458332,1.095664,0.0
4,strategy5,(price_data['fet_open'][MAX_LAG:] + get_lag(pr...,price_data['ng_open'][MAX_LAG:] < numba_indica...,11.0,3.232374e+05,92.353545,6544.844072,107.873304,49.351696,212.673321,0.0
...,...,...,...,...,...,...,...,...,...,...,...
602,strategy603,"np.cos(get_lag(price_data['nvda_low'], lag=5)[...","(np.sin(get_lag(price_data['amzn_high'], lag=2...",625.0,-2.043155e+06,-583.758569,16159.173956,387.562617,3.217634,-435.836956,0.0
603,strategy604,"(get_lag(price_data['nq_volume'], lag=5)[MAX_L...","get_lag(price_data['msft_volume'], lag=5)[MAX_...",735.0,8.114524e+05,231.843535,3451.112859,693.125927,3.142318,169.727264,0.0
604,strategy605,"(np.sin(get_lag(price_data['sui_high'], lag=4)...","(np.sin(get_lag(price_data['cl_high'], lag=4)[...",245.0,1.495194e+05,42.719821,3467.453825,154.878158,3.097268,40.498404,0.0
605,strategy606,((numba_indicators.n_day_high(prices=price_dat...,"(np.cos(get_lag(price_data['aav_low'], lag=1)[...",813.0,4.325441e+05,123.584035,3172.974648,993.886236,3.078844,87.828359,0.0


### Showing performance for strategies that has passed entry and core testing and have positive ROI

In [30]:
final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies)) & 
    (final_perf_df['ROI (%)'] > 0)
]

,strategy,buy,sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),PNL_AVGD_Ratio,Sharpe_Ratio,Not_Working
5,strategy6,"(get_lag(price_data['zf_volume'], lag=2)[MAX_L...","np.sin(get_lag(price_data['sui_open'], lag=2)[...",17.0,35123.405147,10.035259,2337.537958,136.999259,49.066545,58.179906,0.0
57,strategy58,"(np.sin(get_lag(price_data['sui_high'], lag=4)...",np.sin(price_data['es_open'][MAX_LAG:]) < 0.3167,69.0,153173.725735,43.763922,1744.879545,154.837336,44.149633,220.607796,0.0
246,strategy247,"(get_lag(price_data['meta_volume'], lag=3)[MAX...","np.log(get_lag(price_data['pltr_high'], lag=3)...",55.0,249588.313163,71.310947,10314.615947,138.739485,22.221516,128.870141,0.0
248,strategy249,"(get_lag(price_data['meta_volume'], lag=3)[MAX...","np.log(get_lag(price_data['pltr_high'], lag=3)...",55.0,249588.313163,71.310947,10314.615947,138.739485,22.221516,128.870141,0.0
293,strategy294,"((np.cos(get_lag(price_data['lin_open'], lag=1...","np.cos(get_lag(price_data['googl_open'], lag=3...",191.0,204343.444076,58.383841,5579.419705,190.027429,18.454963,75.757811,0.0
324,strategy325,"get_lag(price_data['aapl_volume'], lag=5)[MAX_...","get_lag(price_data['dog_volume'], lag=2)[MAX_L...",33.0,186424.983968,53.264281,10108.091383,130.046377,16.509215,168.509330,0.0
402,strategy403,"(np.sin(get_lag(price_data['sui_high'], lag=4)...",(np.sin(price_data['es_open'][MAX_LAG:]) % np....,17.0,35237.276454,10.067793,1821.244082,160.525547,12.714937,410.021166,0.0
414,strategy415,"get_lag(price_data['amzn_low'], lag=4)[MAX_LAG...","get_lag(price_data['lin_low'], lag=2)[MAX_LAG:...",11.0,92291.016542,26.368862,7446.192395,117.979698,11.987783,194.612854,0.0
418,strategy419,"(get_lag(price_data['6e_volume'], lag=1)[MAX_L...",(price_data['pltr_high'][MAX_LAG:] - get_lag(p...,93.0,202935.960739,57.981703,15883.413646,191.360945,11.699585,71.658168,0.0
431,strategy432,"np.cos(get_lag(price_data['msft_low'], lag=5)[...","(np.sqrt(get_lag(price_data['amzn_close'], lag...",5.0,206860.855910,59.103102,19733.062078,125.912053,11.466610,533.419225,0.0


### Showing performance for strategies that has passed all tests and have positive ROI

In [31]:
final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
    (final_perf_df['ROI (%)'] > 0)
]

,strategy,buy,sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),PNL_AVGD_Ratio,Sharpe_Ratio,Not_Working
5,strategy6,"(get_lag(price_data['zf_volume'], lag=2)[MAX_L...","np.sin(get_lag(price_data['sui_open'], lag=2)[...",17.0,35123.405147,10.035259,2337.537958,136.999259,49.066545,58.179906,0.0
293,strategy294,"((np.cos(get_lag(price_data['lin_open'], lag=1...","np.cos(get_lag(price_data['googl_open'], lag=3...",191.0,204343.444076,58.383841,5579.419705,190.027429,18.454963,75.757811,0.0
486,strategy487,"np.cos(get_lag(price_data['nvda_low'], lag=5)[...","(np.sqrt(get_lag(price_data['amzn_close'], lag...",37.0,226501.051110,64.714586,16500.767274,168.489642,8.914352,155.184056,0.0
493,strategy494,"get_lag(price_data['aapl_volume'], lag=5)[MAX_...","get_lag(price_data['dog_volume'], lag=2)[MAX_L...",7.0,246719.936175,70.491410,27339.828106,115.064512,8.783005,363.186271,0.0


In [42]:
final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
    (final_perf_df['ROI (%)'] > 0)
][[col for col in final_perf_df.columns if col not in ['buy', 'sell']]]

,strategy,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),PNL_AVGD_Ratio,Sharpe_Ratio,Not_Working
5,strategy6,17.0,35123.405147,10.035259,2337.537958,136.999259,49.066545,58.179906,0.0
293,strategy294,191.0,204343.444076,58.383841,5579.419705,190.027429,18.454963,75.757811,0.0
486,strategy487,37.0,226501.051110,64.714586,16500.767274,168.489642,8.914352,155.184056,0.0
493,strategy494,7.0,246719.936175,70.491410,27339.828106,115.064512,8.783005,363.186271,0.0


In [39]:
buy_str_list = final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
    (final_perf_df['ROI (%)'] > 0)
]['buy'].tolist()

print(buy_str_list[3])

get_lag(price_data['aapl_volume'], lag=5)[MAX_LAG:] < get_lag(price_data['aapl_high'], lag=5)[MAX_LAG:]


In [40]:
sell_str_list = final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
    (final_perf_df['ROI (%)'] > 0)
]['sell'].tolist()

print(sell_str_list[3])

get_lag(price_data['dog_volume'], lag=2)[MAX_LAG:] < get_lag(price_data['pltr_open'], lag=2)[MAX_LAG:]


In [43]:
final_entry_win_pc_df.to_csv('testing_results/entry_testing_inst_ind_comb_numba.csv', index=False)
final_exit_win_pc_df.to_csv('testing_results/exit_testing_inst_ind_comb_numba.csv', index=False)
final_core_win_pc_df.to_csv('testing_results/core_testing_inst_ind_comb_numba.csv', index=False)
final_perf_df.to_csv('testing_results/perf_inst_ind_comb_numba.csv', index=False)